In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.output_subarea output_text output_stream output_stdout{width:97%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:14pt;}
div.text_cell_render.rendered_html{font-size:14pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:14px;}
div.output {font-size:14pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:14pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:14pt;padding:5px;}
table.dataframe{font-size:14px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecone
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    (https://www.pinecone.io에서 api key 생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [3]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법_with_table.docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
#    separators=["\n\n","\n"," ",""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

194

In [3]:
document_list[46]

Document(metadata={'source': 'data/소득세법_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의

In [4]:
# embedding : upstage의 solar-embedding-1-large-passage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()   #안하면 403에러
embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

In [8]:
len(embedding.embed_query("소득세법"))

4096

In [5]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# # 데이터를 처음 업로드 할 때 
index_name = "tax-index-table"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )  ##한번만 실행
# 업로드시 경고가 안보이려면 아나콘다 프롬프트 llm환경에서 conda install -c conda-forge ipywidgets

# 업로드한 벡터DB를 가져올때
database = PineconeVectorStore(
    embedding=embedding,  # 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

CPU times: total: 7.92 s
Wall time: 1min 11s


# 2. 답변 생성전 retrieval 확인

In [6]:
query = "연봉이 5천만원인 직장인의  소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3)  # 기본 k값:4

In [11]:
retrieved_docs[1]

Document(id='195be775-d741-487f-8f63-7f266953d986', metadata={'source': 'data/소득세법_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n제2관 세액공제 <개

In [12]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [13]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrieved_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [16]:
retrieved_docs[0]

Document(id='004b3092-a41d-4cd0-9349-12f989f06681', metadata={'source': 'data/소득세법_with_table.docx'}, page_content='1. 총급여액이 3천 300만원 이하인 경우: 74만원\n\n2. 총급여액이 3천 300만원 초과 7천만원 이하인 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]. 다만, 위 금액이 66만원보다 적은 경우에는 66만원으로 한다.\n\n3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는 경우: 50만원 - [(총급여액 - 1억2천만원) × 1/2]. 다만, 위 금액이 20만원보다 적은 경우에는 20만원으로 한다.\n\n③ 일용근로자의 근로소득에 대해서 제134조제3항에 따른 원천징수를 하는 경우에는 해당 근로소득에 대한 산출세액의 100분의 55에 해당하는 금액을 그 산출세액에서 공제한다.<개정 2014. 1. 1.>\n\n[전문개정 2012. 1. 1.]\n\n\n\n제59조의2(자녀세액공제) ①종합소득이 있는 거주자의 기본공제대상자에 해당하는 자녀(입양자 및 위탁아동을 포함하며, 이하 이 조에서 “공제대상자녀”라 한다) 및 손자녀로서 8세 이상의 사람에 대해서는 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다. <개정 2015. 5. 13., 2017. 12. 19., 2018. 12. 31., 2019. 12. 31., 2022. 12. 31., 2023. 12. 31., 2024. 12. 31.>\n\n1. 1명인 경우: 연 25만원\n\n2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한

# 3. 답변 생성

In [17]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system","content":"당신은 최고의 한국 소득세법 전문가입니다"},
        {"role":"user",
         "content":f""""[context]를 참고해서 사용자의 질문에 10줄이내로 답변해 주세요
-[context]:{retrieved_doc}
-질문:{query}"""}
    ],
    temperature=0.2
)

In [19]:
print(response.choices[0].message.content)

연봉이 5천만원인 직장인의 소득세를 계산하기 위해 먼저 종합소득 과세표준을 확인해야 합니다. 

1. **종합소득 과세표준**: 5천만원
2. **세율 적용**:
   - 1,400만원 이하: 6% → 84만원
   - 1,400만원 초과 5,000만원 이하: 84만원 + (5,000만원 - 1,400만원) × 15% = 84만원 + 525만원 = 609만원

따라서, 5천만원의 연봉을 가진 직장인의 소득세는 **609만원**입니다.


In [22]:
from openai import OpenAI
import os
# upstage의 과금정책 : https://www.upstage.ai/pricing/api
client = OpenAI(
    api_key=os.getenv("UPSTAGE_API_KEY"),
    base_url="https://api.upstage.ai/v1"
)
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {"role":"system","content":"당신은 최고의 한국 소득세법 전문가입니다"},
        {"role":"user",
         "content":f""""[context]를 참고해서 사용자의 질문에 10줄이내로 답변해 주세요
-[context]:{retrieved_doc}
-질문:{query}"""}
    ],
    temperature=0.2
)

In [23]:
print(response.choices[0].message.content)

연봉 5천만원인 직장인의 소득세는 다음과 같이 계산됩니다(2024년 기준, 근로소득세액공제 적용):

1. **종합소득산출세액 계산**  
   - 5,000만원 × 15% - 126만원 = **624만원**  
     (1,400만원 이하 6% + 초과분 15% 적용)

2. **근로소득세액공제 적용**  
   - 5,000만원(총급여액)이 3,300만원 초과 7,000만원 이하 구간:  
     **74만원 - (5,000만원 - 3,300만원) × 8/1,000 = 66만원**  
     (최소 66만원 적용)

3. **최종 산출세액**  
   - 624만원 - 66만원 = **558만원**  
   (단, 지방소득세 10% 추가 시 총 613.8만원)

※ 실제 납부세액은 보험료·의료비 등 추가 공제, 자녀세액공제(제59조의2) 등에 따라 달라질 수 있습니다.


In [11]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [12]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" # 느리지만 더 깊게 추론함(low, medium)
)

In [13]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
[context]를 참고해서 사용자의 질문에 답변해주세요
[context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [14]:
ai_message = llm.invoke(prompt)

In [15]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세 계산은 다음과 같은 단계로 진행됩니다. 단, **자녀세액공제 및 기타 추가 공제는 적용되지 않는다고 가정**하고, **2024년 기준 현행 법령**을 적용합니다.

---

### **1. 근로소득공제 계산 (제47조)**
- **총급여액**: 5,000만원  
- **근로소득공제율** (2024년 기준):  
  - 1,500만원 이하: 70%  
  - 1,500만원 초과 ~ 4,500만원 이하: 40%  
  - 4,500만원 초과 ~ 1억원 이하: 15%  
  - 1억원 초과: 10%  

**계산**:  
- 1,500만원 × 70% = **1,050만원**  
- (4,500만원 - 1,500만원) × 40% = **1,200만원**  
- (5,000만원 - 4,500만원) × 15% = **75만원**  
- **총 근로소득공제액** = 1,050 + 1,200 + 75 = **2,325만원**  
  (단, 최대 2,000만원 한도 적용 → **2,000만원** 공제)

---

### **2. 과세표준 계산**
- **과세표준** = 총급여액 - 근로소득공제  
  = 5,000만원 - 2,000만원 = **3,000만원**

---

### **3. 종합소득산출세액 계산 (누진세율 적용)**
2024년 종합소득세 세율 (과세표준 기준):  
| 과세표준 구간 | 세율 | 누진공제 |
|---------------|------|----------|
| 1,200만원 이하 | 6% | - |
| 1,200만원 초과 ~ 4,600만원 이하 | 15% | 108만원 |
| 4,600만원 초과 ~ 8,800만원 이하 | 24% | 522만원 |

**계산**:  
- 3,000만원 × 15% - 108만원 = **450만원 - 108만원 = 342만원**

---

### **4. 근로소득세액공제 적용 (제59조)**
- **총급여액 5,000만원**은 **3,300만원 초과 ~ 7,000만원 이하** 구간에 해당합

# 4. langchain 답변 생성
- ch9.07_LangChain과 vectorDatabase를 활용한 RAG구현(UpstageEmbedding) 참조

In [24]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
# 1. LLM과 임베딩 초기화
load_dotenv()
# llm = ChatOpenAI(model="gpt-4.1-mini")
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")
                  
embedding = UpstageEmbeddings( model="solar-embedding-1-large-passage")

# 2. 업로드한 벡터db 가져올 때
vectorstore = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
# vectorstore.similarity_search("질문", k=2) - 어떤 질문이 들어올지 몰라서 쓸수 없다.

# 4.프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return "\n\n---\n\n".join([doc.page_content for doc in documents])

In [25]:
# 6. RAG 체인 구성(LCEL 방식)
from langchain_core.runnables import RunnablePassthrough  # {"query":"~"}  => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough()  # 질문 그대로 전달
    }
    | prompt  # prompt에 context와 query 변수 주입
    | llm     # llm에 
    | StrOutputParser()
)
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
rag_chain.invoke(query)

'주어진 문맥에 따르면, 연봉 5천만원(총급여액 7천만원 이하)인 직장인의 근로소득세액공제는 **66만원**입니다(제59조 ②항 2호). 단, 이는 근로소득세액공제액이며, 실제 소득세는 종합소득과세표준에 따른 세율(제55조)과 추가 공제(자녀세액공제 등)를 고려해야 정확한 계산이 가능합니다. 제공된 정보만으로는 산출세액을 확정할 수 없습니다.'